import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
df = pd.read_csv("Dataset .csv")

# Keep only needed columns
df = df[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'City']]
df.dropna(inplace=True)

# Reset index for safety
df.reset_index(drop=True, inplace=True)

df.head()


,Restaurant Name,Cuisines,Average Cost for two,City
0,Le Petit Souffle,"French, Japanese, Desserts",1100,Makati City
1,Izakaya Kikufuji,Japanese,1200,Makati City
2,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4000,Mandaluyong City
3,Ooma,"Japanese, Sushi",1500,Mandaluyong City
4,Sambo Kojin,"Japanese, Korean",1500,Mandaluyong City


In [4]:
# Convert all cuisines to lowercase
df['Cuisines'] = df['Cuisines'].str.lower()

# Use TF-IDF to vectorize cuisines
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Cuisines'])

# Compute similarity between all restaurants
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [7]:
def recommend_restaurants(restaurant_name, top_n=5):
    # Convert restaurant name to lowercase
    restaurant_name = restaurant_name.lower()

    # Get all matching restaurant indexes
    matching = df[df['Restaurant Name'].str.lower() == restaurant_name]

    if matching.empty:
        print("Restaurant not found.")
        return []

    idx = matching.index[0]  # Just take the first match

    # Compute similarity
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort and get top results
    sim_scores = sorted(sim_scores, key=lambda x: float(x[1]), reverse=True)

    # Skip the input restaurant itself (first one)
    sim_scores = sim_scores[1:top_n+1]

    # Get restaurant indices
    restaurant_indices = [i[0] for i in sim_scores]

    # Return restaurant names
    return df['Restaurant Name'].iloc[restaurant_indices]


In [8]:
# Try recommending restaurants similar to one you like
recommend_restaurants("Domino's Pizza")


,Restaurant Name
32,Rovereto
151,Flying Pie Pizzaria
153,Guido's Original New York Style Pizza
176,A & A Pagliai's Pizza
199,Mellow Mushroom


In [9]:
restaurant_input = input("Enter a restaurant name you like: ")
recommendations = recommend_restaurants(restaurant_input)
print("Recommended restaurants:\n")
print(recommendations)


Enter a restaurant name you like: Flying Pie Pizzaria
Recommended restaurants:

32                                  Rovereto
151                      Flying Pie Pizzaria
153    Guido's Original New York Style Pizza
176                    A & A Pagliai's Pizza
199                          Mellow Mushroom
Name: Restaurant Name, dtype: object
